# Google Earth Engine Python (GEE) API Tutorial
By: Richard Udell

The tutorial is presented as a walk-through of an analysis: **Identifying Urban Trend Surrounding Denver International Airport (DIA) From 1995 to 2012**.  
# Goal
The goal of this notebook is to introduce new Python users to a seemingly complex combination of tools - Google Earth Engine, Jupyter Notebook, and the Python programming language.  

#### What has kept me motivated throughout the development of this tutorial?
I am a new Python user and an even newer GEE user and I have overcome **some** of the challenges of using the Google Earth Engine Python API in Jupyter Notebooks and I want to make it easier for the next user.

# Relevance
As the name implies, the Google Earth Engine Python API combines two tools: Python and Google Earth Engine.  Google Earth Engine has an [easy-to-use online code editor](https://code.earthengine.google.com) that includes the documentation and a [GEE guide](https://developers.google.com/earth-engine#background) that is very helpful for new-users.  

However, there are two problems for users of the GEE Python API:
1. The GEE documentation and the GEE guide are in JavaScript
2. There are few resources for users that choose to run the GEE Python API on their own Python environment inside Jupyter Notebooks

#### Why to use the Google Earth Engine Python API?
The Python programming language is [very popular for good reasons](https://www.python.org/about/) and has an active open-source community that actively maintains many powerful packages for data-analysis.

Google Earth Engine is a very powerful tool for two reasons: 
1. GEE provides easy access to very large spatial datasets from organizations all over the world via the [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets) without ever having to store them locally on your computer - many of these spatial datasets are much larger than your computers hard drive.  
2. GEE provides access to a much more powerful computer.  Some of the calculations that you will perform in GEE would take days to do on your computer’s processor, but GEE servers will do the calculations for you and return the results in seconds.


The combination of Google Earth Engine and Python can increase the efficiency, reach, and speed of very large-scale (I'm talking pedabytes!) spatial data-analysis.  

# Overview of Datasets
To keep the workflow simple we will work with a single dataset: [Landsat 5](https://www.usgs.gov/land-resources/nli/landsat/landsat-5?qt-science_support_page_related_con=0#qt-science_support_page_related_con).  Landsat 5 was in operation from 1984 to 2012.  It collected 30m resolution data from the entire Earth at a period of 16 days.

# Overview of Approaches/Methods
The tutorial covers the steps to **make a linear fit model that identifies the trend in urban land cover surrounding the Denver International Airport (DIA) from 1995 to 2012 with the Google Earth Engine Python API**.  However, the tutorial completes once the linear fit model has been visualized on the map.  The methods used to interepret of the results are not included in the tutorial.  

The results are interpretted by exporting the linear fit model from GEE to a .TIF file then re-importing the .TIF file to be analyzed with Python tools such as numpy.

# Google Earth Engine Python API Tutorial: 
### Identifying Urban Trend Surrounding Denver International Airport (DIA)
By: Richard Udell

# Table of Contents
    Setup
    Introduction
1. The Interactive Map
2. Image Collections (`ee.ImageCollection` Objects)
3. Overcoming JavaScript Documentation
4. Linear Fit Workflow Walk Through

# Setup

Be sure you have [applied for Earth Engine](https://signup.earthengine.google.com/) and have [geemap installed](https://github.com/giswqs/geemap#installation)  on your computer.

# Introduction
In this tutorial we will make a linear fit model that identifies the trend in urban land cover surrounding the Denver International Airport (DIA).

It is assumed that you have completed introductory tutorials in Python and have previously used Jupyter Notebooks.

##### Important note:  be sure to restart and clear the kernel before you begin the tutorial.  Run the code cells as you progress.

# Tutorial Part 1: The Interactive Map
1. Import Necessary Packages
2. Initiate the interactive map
3. Define Region of Interest
    * Preset ROI
    * Draw Your Own ROI
4. Understanding `ee` Objects
    * Visualizing `ee` Objects
    * Examining `ee` Objects
5. Exercise: Save the Coordinates of `roi`

## 1.1 Import Necessary Packages
**Without geemap we would not be able to visualize our code.**  Without geemap working with the Google Earth Engine Python API would be very difficult.  

In [1]:
# Import necessary packages
import os
import numpy as np
import matplotlib.pyplot as plt

import rasterio as rio
import earthpy as et

import ee
import geemap

## 1.2 Initiate the Interactive Map

In [2]:
# Initiate interactive map with the name Map, default basemap is HYBRID
Map = geemap.Map(center=(39.791424, -104.809968), zoom=11)

# To see a list of basemaps to choose from
# geemap.ee_basemaps.keys()

# Change the basemap if you'd like
Map.add_basemap('HYBRID')  

Now that you have initiated the interactive map and named it `Map`, you can view it by calling `Map`.

Note: The Map object will automatically be updated whenever you add new layers.  You do not need to call the Map again to update it (even if the new layer was added in a cell that was run later than the `Map` was called).

In [3]:
# View the interactive map by calling the Map object
Map

Map(center=[39.791424, -104.809968], controls=(WidgetControl(options=['position'], widget=HBox(children=(Toggl…

## 1.3 Define Region of Intersest (ROI)
The first step in our Linear Fit workflow is to define the region of interest (ROI).  

This can be done by defining the radius of a cirlce around a point by creating an `ee.Geometry.Point` object and selecting a buffer (in meters).  This is shown in option 1 below.

You could also create an `ee.Geometry.Polygon` object and pass a `list` of coordinates.  This is shown in option 2 below.

#### Preset ROI

In [4]:
# Option 1:
# Set ROI as 10 mile (16,093 meters) radius around the intersection of
# Pena Blvd and E-470 (about the SW corner of the airport)
roi = ee.Geometry.Point([-104.746984, 39.834241]).buffer(16093)

# Option 2:
# Set ROI as a Polygon object with your own coordinates
# (uncomment the code below to run this instead)
# roi = ee.Geometry.Polygon([[-104.90118, 39.74158],
#                            [-104.90118, 39.913489],
#                            [-104.628571, 39.913489],
#                            [-104.628571, 39.74158],
#                            [-104.90118, 39.74158]])

#### Draw Your Own ROI
Alternatively, you can draw a shape on the interactive Map above then extract that shape as an `ee.Geometry.Polygon` object.  This is done by first drawing the shape on the Map above then **uncommenting line 2 in the code cell below** and running the code.  ![Draw Features on Map](images/draw_features_geemap.png)

In [5]:
# # Draw your own ROI (polygon) on the Map then save it to a variable
# roi = ee.FeatureCollection(Map.draw_features)

## 1.4 Understanding `ee` Objects
*Congratulations!*  You have created your first `ee` object.  Whenver you have an `ee` object you can view its section in the Google Earth Engine documentation to understand what methods can be run on this object.  

There are seven different types of `ee.Geometry` objects, however, in this tutorial we will be working with the `ee.Geometry.Polygon` object. 

Note: When you draw a shape on the interactive map, even if it's a circle, it will be stored as a `Polygon` object. 

### Visualizing `ee` Objects
Visualize your `ee.Geometry.Polygon` object by adding your first layer to the Map.  Here is the code: 

`Map.addLayer(eeObject, visParams, name, shown, opacity)`

Adding a layer to the Map is something that you will do often.  In the case of your ROI, we will leave the `visParams` argument empty because we do not need to select bands or give any minimum, maximum or gain values because we simply want a black shape on our map.


*See the [GEE docs](https://code.earthengine.google.com) to view the documenataion on the `Map.addLayer()` function in more detail.*  


In [6]:
# Add your ROI to the map as your first map layer
Map.addLayer(roi, {}, 'ROI', opacity=0.5)

*Awesome!* 
Now you have added your first map layer.  Remember, although the basemap is a layer, it hardly functions as one because it does not hold any data.  Use the floating box on the right-hand side to toggle layers on and off. ![Layer floating box](images/layer_floating_box.png)

### Examining `ee` Objects
Let's talk about object types when you are using the Google Earth Engine Python API.  In the code below we will examine the `roi` variable as we would with any variable in Python.  It is not similar to a normal python object because it will not print out its contents.  

In [7]:
# Examine type
print('roi type is: ', type(roi), '\n')

# Try to print roi
print('roi print output is:\n\n', roi)

roi type is:  <class 'ee.geometry.Geometry'> 

roi print output is:

 ee.Geometry({
  "type": "Invocation",
  "arguments": {
    "geometry": {
      "type": "Point",
      "coordinates": [
        -104.746984,
        39.834241
      ]
    },
    "distance": 16093
  },
  "functionName": "Geometry.buffer"
})


##### Are You Confused?  `.getInfo()` Might Help!
Rather than printing out its contents, `print(roi)` has only confused us.

One method that you can run on almost any `ee` object is `.getInfo()`.  You should run `.getInfo()` in any circumstance when you would otherwise run `print()` to attempt to look inside an object; it will return a dicionary that holds valuable metadata including the object `type`.  If `.getInfo()` does not provide the understanding you need then remember you can always try visualizing the `ee` object with `Map.addLayer()`.

In [8]:
# Look inside of the roi variable
roi.getInfo()

{'type': 'Polygon',
 'coordinates': [[[-104.74698400000003, 39.97905974975719],
   [-104.79976678659239, 39.973284016955745],
   [-104.84832310085373, 39.95641936004324],
   [-104.88877075086636, 39.9298157045635],
   [-104.91788620683361, 39.89560054511281],
   [-104.93336237331994, 39.856506506520816],
   [-104.93398889066185, 39.815650984223154],
   [-104.91974170352333, 39.77628629385638],
   [-104.89177691750443, 39.74154074105805],
   [-104.85233194858966, 39.714171221997894],
   [-104.80454392948539, 39.696346534149995],
   [-104.75220087244183, 39.68947778599627],
   [-104.69944509491718, 39.69410846624248],
   [-104.65045098033968, 39.70987219021903],
   [-104.60910042725811, 39.73552117924187],
   [-104.57867945211666, 39.769023414890334],
   [-104.56161836526535, 39.807721398184675],
   [-104.55929563793609, 39.848540796379055],
   [-104.57192186421288, 39.88823326719261],
   [-104.5985149854282, 39.9236347303136],
   [-104.6369712428352, 39.9519186347889],
   [-104.68422849

## Exercise: Save the Coordinates of `roi`
In this exercise you will save the coordinates of the `ee.Geometry.Polygon` you drew on the interactive Map.

If you have not already done this then return to section **Draw Your Own ROI** in **Section 1.3 Define Region of Interest (ROI)**.


It is useful to save the coordinates of the ROI that you drew because you will have to redraw the ROI each time you restart and clear the kernel.  

To do this we must extract the coordinates from the dictionary that is returned when we call `.getInfo()` and create an `ee.Geometry.Polygon` object with those coordinates.  This must be done with good old fashioned copy/paste.  **First, uncomment the code below**, then run the code!

In [9]:
# # Navigate through the dictionary to extract the coordinates
# coords = roi.getInfo()['features'][0]['geometry']['coordinates'][0]
# coords

In [10]:
# # Copy the output from the previous cell and paste the list as the argument
# for the ee.Geometry.Polygon(arg) below
# roi = ee.Geometry.Polygon(# list of coordinates here)

# Tutorial Part 2: Image Collections (`ee.ImageCollection` Objects)
1. Loading Image Collections
2. Filtering Image Collections

## 2.1 Loading Image Collections
Working with ee.ImageCollections looks really simple but is actualy really powerful.

Visit the Google Earth Engine Data Catalog (https://developers.google.com/earth-engine/datasets) to select the Landsat 5 dataset.  Save this dataset into a variable named `all_Landsat5`.  

*Note: this is called "loading" an Image Collection rather than "importing".  This is because you are not "importing" any data to your local computer, instead you are establishing a connection with the Image Collection on the Google servers.*

In [11]:
# The entire Landsat5 dataset
all_Landsat5 = ee.ImageCollection('LANDSAT/LT05/C01/T1')

# Examine object type
print('all_Landsat5 object type: ', type(all_Landsat5))

# How many ee.Images are in the ee.ImageCollection
# (this may take a minute to run)
print('\nTotal number of Landsat scenes (ee.Image objects) in the collection: ',
      all_Landsat5.size().getInfo())

all_Landsat5 object type:  <class 'ee.imagecollection.ImageCollection'>

Total number of Landsat scenes (ee.Image objects) in the collection:  1657727


It only took one line of code, but `all_Landsat5` now holds every single Landsat scene ever taken in the history of the Landsat5 sattellite.  Each Landsat scene is stored in GEE as an `ee.Image` object, so the entire Landsat 5 dataset is stored as a single `ee.ImageCollection` object that holds of all of those individual `ee.Image` objects.  As shown in the code above, it's pretty incredible that there are **over one and a half milliion Landsat scenes in `all_Landsat5`**.  

## 2.2 FIltering Image Collections
The power of the Google Earth Engine Python API is that you can filter through all of those images without ever having to store them locally on our computer.

You can use `.filterDate()` and `.filterBounds()` to filter through the collection.  Let's filter our collection to only include ee.Images that cover our `ee.Geometry.Polygon` object, `roi`, and have been taken since the completion of the construction of DIA (1995).  

In [12]:
# Filterting by only date reduces the collection to just over one million
# ee.Image objects (this may take a minute to run)
print('Size of collection (1995-2012): ',
      all_Landsat5.filterDate('1995-01-01', '2012-12-31').size().getInfo())

# Filter by date and bounds and save collection to a new variable
filtered_Landsat5 = all_Landsat5.filterDate(
    '1995-01-01', '2012-12-31').filterBounds(roi)

# Filtering by date and bounds will reduce the collection by a much larger degree
print('Size of collection (1995-2012) and filtered by ROI: ',
      filtered_Landsat5.size().getInfo())

Size of collection (1995-2012):  1047796
Size of collection (1995-2012) and filtered by ROI:  938


# Tutorial Part 3: Overcoming JavaScript Documentation
1. Github Repository of Translated Scripts
2. Translating From JavaScript to Python Yourself
    * JavaScript example
    * Python Translation


One challenge you'll face when working with the GEE Python API is that the [GEE documentation](https://code.earthengine.google.com) and [GEE guide](https://developers.google.com/earth-engine) are in JavaScript but you need Python!

When you find a code example in the [GEE documentation](https://code.earthengine.google.com) that you want to use in your Python workflow you have two options:
* Option 1: Visit the [translated GEE documentation on Github](https://github.com/giswqs/earthengine-py-notebooks) and navigate the repository to see if the same code example exists in the Python translation. 
* Option 2: Translate the code yourself!

## 3.1 Github Repository of Translated Scripts



##### [Dr. Qiusheng Wu](https://geography.utk.edu/about-us/faculty/dr-qiusheng-wu/) is a Geography Professor at the University of Tennessee Knoxville
In addition to creating the geemap Python package, he [mostly translated the GEE documentation from JavaScript to Python](https://github.com/giswqs/earthengine-py-notebooks).


## 3.2 Translating From JavaScript to Python Yourself

The following two code cells contain the same code, however the first is in JavaScript and the second is in Python.  It may seem cumbersome, but the reality when working with Google Earth Engine Python API is that it is often quicker to translate the code in the [GEE guide](https://developers.google.com/earth-engine) yourself.  Although this is tedious, it may prove to help  you understand your code and stop you from instinctively copying/pasting code snippets from the guide.

Here are some guidelines for translating code from JavaScript to Python:
* Remove the `var`'s and `;`'s and you're halfway there!
* Edit dictionary keys
    * In JavaScript a dictionary key will not be in quotes, however in Python a dictionary key needs to be a string.
    * Lookout for visualization parameters, they are typically entered as a dictionary
* Reformat Functions
    * Python functions do not need to be within brackets and are prefixed with `def`
    * The first line of a Python function needs to end with `:`
    * Be sure to re-indent the content of the function
* Fix the comment notation - change `//` to `#`

#### JavaScript example 
(Running this code in Jupyter would return a SyntaxError)

    // This function adds a band representing the image timestamp
    var addTime = function(image) {
      return image.addBands(image.metadata('system:time_start'));
    };

    // Define a point on Map
    var point = ee.Geometry.Point([-104.90118, 39.74158]);

    // Load Landsat 5 Image Collection
    var all_Landsat5 = ee.ImageCollection('LANDSAT/LT05/C01/T1');

    // Filter Landsat 5 collection by date and point
    var filtered_Landsat5 = all_Landsat5.filterDate(
        '1995-01-01', '2012-12-31').filterBounds(point).map(addtime);

    // Add collection as a Map Layer
    Map.addLayer(filtered_Landsat5, {
                 bands: ['B3', 'B2', 'B1'], min: 0, max: 128}, 'RGB_Landsat5');
                 
#### Python Translation

    # This function adds a band representing the image timestamp
    def addTime(image):
        return image.addBands(image.metadata('system:time_start'))

    # Define a point on Map
    point = ee.Geometry.Point([-104.90118, 39.74158])

    # Load Landsat 5 Image Collection
    all_Landsat5 = ee.ImageCollection('LANDSAT/LT05/C01/T1')

    # Filter Landsat 5 collection by date and point
    filtered_Landsat5 = all_Landsat5.filterDate(
        '1995-01-01', '2012-12-31').filterBounds(point).map(addtime)

    # Add collection as a Map Layer
    Map.addLayer(filtered_Landsat5, {
                 'bands': ['B3', 'B2', 'B1'], 'min': 0, 'max': 128}, 'RGB_Landsat5')

# Tutorial Part 4: Linear Fit Workflow Walk Through
1. Analysis Approach Explained
2. Recap of Tutoral Part 1 and Part 2
    * Define ROI
    * Load Image Collection
3. Defining Functions for `ee.ImageCollection` Objects
    * Spectral Unmixing Explained
4. Mapping Functions Over `ee.ImageCollection` Objects
5. Linear Fit Reducer

## 4.1 Analysis Approach Explained
The output of this analysis is an ee.Image object that has two bands (meaning each pixel will have two values) that represent a linear fit model.  The first band will be the *scale* and the second value will be the *offset*.  *Scale* and *offset* represent the **slope** and **intercept** of a linear model that has been fit to each pixel of for data that was input to the model. (Recall y = mx+b.)

| Linear Model Output  | Interpretation |
|---|---|
| Scale  |  Slope (m) |
|  Offset |  Intercept (b) |

The goal of this analysis will be to take a collection of Landsat 5 images and **unmix** the band values from wavelength to endmember fractions by a process called spectral unmixing.  This will be done with the built-in GEE `.unmix()` method for `ee.Image` objects (explained in *Section 4.3*).  Once the Landsat5 collection has been processed to contain just four bands (urban, vegetation, water, and a time band) a linear line will be fit to the change in data in each pixel from 1995 to 2012.  This **linear fit** will measure the relationship between the independent variable (time) and the dependent variable (urban).

## 4.2 Recap of Tutoral Part 1 and Part 2

#### Define ROI
In Part 1 we defined the ROI and saved it to a variable called `roi`.  Call `roi` in the cell below to be sure that it contains an `ee.Geometry` object.

In [13]:
roi

#### Load Image Collection
Then in Part 2 we loaded the Landsat 5 collection and saved it to a variable called `all_Landsat5`.  Then we filtered by date (1995-2012) and by bounds (roi) and saved it to a variable called `filtered_Landsat5`.  Call `filtered_Landsat5` in the cell below to be sure that it conatins an `ee.ImageCollection` object.

In [14]:
filtered_Landsat5

## 4.3 Defining Functions for `ee.ImageCollection` Objects

In the code cell below we will define the only function used for our analysis.  This function will be applied to the `ee.ImageCollection` object we have made called `filtered_Landsat5`.  However, we will not apply this function in the same way that you normally run a function in Python. 

Normal Python function:
    
    def function(input):
        return input + 10
        
    number = 1
    new_number = function(number)
    
    print(new_number)
    > 11
    
What we want to do is change every single `ee.Image` object that is inside of the `ee.ImageCollection` object.  This will be done by [*mapping* the function over the entire image collection](https://developers.google.com/earth-engine/ic_mapping).  (Think iterating a function over a list of objects.)

**So**, we can write the function in the same way we normally would write a function in Python **but there is one important distinction**: the input **and** return has to be an `ee.Image` object.  Yes, that means that **the function can only accept one parameter**.

Below is our function, notice taht it accepts only a single `ee.Image` object and returns a single `ee.Image` object.

In [15]:
def unmix_L5(image_L5):
    """Unmix each pixel with the given endmembers, by computing the pseudo-inverse
    and multiplying it through each pixel. Returns an image of doubles with the same
    number of bands as endmembers. (Description from GEE docs).

    Parameters
    ----------
    image_L5 : ee.Image
        Any ee.Image object from Landsat 5 (must include at least the first 7 bands).

    Returns
    ------
    unmixed_image : ee.Image
        A 4-band ee.Image object.  Three bands, ('urban', 'veg', and 'water') are the
        per-pixel fraction that the .unmix() method determined that correspond with
        the three given endmember values.  The final band, 'system:time_start' is
        the time metadata associated with that image.
    """
    # Select bands 1-7
    bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
    image = image_L5.select(bands)

    # Endmembers taken from GEE unmix() example script found here:
    # https://developers.google.com/earth-engine/image_transforms
    urban = [88, 42, 48, 38, 86, 115, 59]  
    veg = [50, 21, 20, 35, 50, 110, 23]  
    water = [51, 20, 14, 9, 7, 116, 4]  
    
    # Run the GEE unmix() method with to get per-pixel fractions that are non-negative
    # and sum-to-one
    unmixed_image = image.unmix([urban, veg, water], True, True).rename([
        'urban', 'veg', 'water'])
    
    # Add time band and return image - time stamp is divided by large number to avoid 
    # extra high (or low) scale values.
    return unmixed_image.addBands(image.metadata('system:time_start').divide(1e13))

#### Spectral Unmixing Explained
So now you may be wondering, "what the heck does this function do?"

As stated in *Section 4.1*, the function takes an ee.Image and unmixes the band values from wavelength to endmember fractions by a process called spectral unmixing. This is be done with the built-in GEE .unmix() method for ee.Image objects. The output is a Landsat5 ee.Image no longer contains its seven bands of reflectances for each wavelength category measured by Landsat 5, these values at each pixel have been "unmixed" to contain a guess as to the fraction of land cover at each pixel for each of the given endmembers (urban, veg, and water).  Additionally, a band that contains a time stamp has been included in the output. 

![Graphic that attempts to explain spectral unmixing in the context of this workflow](images/spectral_unmixing_graphic.png)


An output pixel has one band for each endmember.  The value for each band is the fraction of the area of that pixel that was calculated to correspond with the endmember.  This is interpreted to be the fraction of land cover at that pixel. 

![An example output pixel from spectral unmixing and its interpretation in the context of this workflow.](images/example_output_pixel_interpretation.png)

*If you want an oversimpilified explanation, consider this:
Spectral unmixing can theoretically determine whether the land at each pixel is one of three categories: urban, vegetation, or water.  Each pixel in a Landsat image has a number that basically tells you how bright that small piece of Earth is.  Spectral unmixing considers how bright a piece of Earth should be if it is urban, vegetation, or water and calculates which category each pixel fits into.  However, it’s a bit more complicated - spectral unmixing assigns a percentage of each category to each pixel.  For example: if you examine a Landsat pixel in a city park next to a pond, there could be 50% urban, 25% vegetation and 25% water in the 30m x 30m pixel.*



## 4.4 Mapping Functions Over `ee.ImageCollection` Objects

This part of the workflow is really simple!  Again, GEE Python API looks so simple, but it's pretty incredible to think that this function is running on every single image in the collection.  If you're using the *Preset ROI, Option 1* from *Section 1.3* (10 mile radius around the airport) then this ImageCollection holds 938 images!  

Remember, we are changing every single Image in the ImageCollection by mapping the function over the entire image collection with the `.map()` method.  To learn more reveiw the [section on Mapping Over ImageCollections in the GEE Guide](https://developers.google.com/earth-engine/ic_mapping).

*Note about runtime: one thing you will notice is that this code runs surprisingly quickly!  However, the code is not actually being run on the Google servers until you request an output (like visualizing with `Map.addLayer()` or `.getInfo()`)* 

In [16]:
# Apply unmix_L5 function to each Image in the ImageCollection by "mapping" it
# over the ImageCollection
unmixed_Landsat5 = filtered_Landsat5.map(unmix_L5)

## 4.5 Linear Fit Reducer

Now we have processed our `ee.ImageCollection` object to the point that we can do some analysis!  Our ImageCollection now has four bands: `'urban'`, `'veg'`, `'water'`, and `'system:time_start'`.  Remember, we are using Landsat imagery from across 17 years to see if there is a trend in urban land cover fraction at each pixel.  Therefore, our independent variable is time (represented by the band `'system:time_start'`) and our dependent variable is the urban fraction at each pixel (represented by the band `'urban'`).

To learn more about the `linearFit()` function and Reducers in general in Google Earth Engine, review the [Linear Regression section in the GEE Guide](https://developers.google.com/earth-engine/reducers_regression).

In [17]:
# Reduce the collection with the ee.Reducer, linearFit
# Independent variable are followed by dependent variables.
linearFit = unmixed_Landsat5.select(
    ['system:time_start', 'urban']).reduce(ee.Reducer.linearFit())

It's that simple!  We have done it!  We have fit a linear model that measures trend in the fraction of urban at each pixel over time from 1995 to 2012. 

To be sure that we got what we were expecting (an ee.Image object with two bands (`'scale'` and `'offset'`) let's look inside our linearFit variable with `.getInfo()` as we learned earlier in this tutorial.

In [18]:
# Verify that it is an ee.Image object with two bands (scale and offset)
linearFit.getInfo()

{'type': 'Image',
 'bands': [{'id': 'scale',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'offset',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

Now, let's visualize our linear model.  First, we will clip the model to the ROI, then add the model as a new layer to the interactive map.  Then we will call the interactive map object, `Map`, so that we do not have to scroll all the way to the top of the notebook (where `Map` was last called) to view the new layer.

In [19]:
# Clip ee.Image to ROI
clipped_linearFit = linearFit.clip(roi)

# Add linearFit model to the interactive map
Map.addLayer(clipped_linearFit,
             {'min': 0, 'max': [-0.9, 8e-5, 1], 'bands': ['scale', 'offset', 'scale']}, 'Linear_Fit')

# Add legend to map
legend_keys = ['High Negative Urban Trend', 'Negative Urban Trend',
               'Nearly No Urban Trend', 'Positive Urban Trend']
legend_colors = ['#0000FF', '#01FFFF', '#00FF1F', '#FFFF02']

Map.add_legend(legend_keys=legend_keys,
               legend_colors=legend_colors, position='bottomleft')

# Display the interactive map
Map

Map(center=[39.791424, -104.809968], controls=(WidgetControl(options=['position'], widget=HBox(children=(Toggl…

##### Interpretation of plot: 
The yellow pixels corresond to areas in the ROI that havea  positive urban trend.  If you turn off the 'Linear_Fit' layer, the basemap reveals high quality imagery of the land as it appears today, in 2020.  Comparing the areas in yellow, that are calculated to have had a positive urban trend in 2012, to the basemap, it appears that the yellow areas align with neighborhoods that exist today.  However, there are few areas of the map that are extensively in yellow.  On the other hand, the majority of the map marked as having a negative urban trend (light blue).  

**Congratulations!**  You have completed the tutorial!  Now you are prepared to start using the Google Earth Engine Python API to explore spatial data.  A great next step is to pick a new dataset from the [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets) and import is as an `ee.ImageCollection` and filter the collection in unique ways!  

# Results/Conclusions of Analysis
In order to interpret the results of the linear fit model, I have exported the model form GEE (as a .TIF file) then re-imported the model into a numpy array.  The linear fit model is interpreted in the code cells below to determine that 13% of the pixels in the ROI have a positive urban trend.  This means that the urban fraction at each of those pixels has increased throughout the temporal range of the analysis (1995 to 2012). 

#### Conclusions of Tutorial
The primary limitation for a new user to the Google Earth Engine Python API is that the GEE documentation and guide are in JavaScript.  I believe that it is imperative that new users learn to read JavaScript because the guide is a very useful resource.  However, I do not believe that this is a large limitation because JavaScript and Python are very similar languages.

It is also clear that using using the GEE Python API in Jupyter Notebooks would be much more difficult without the use of geemap. 

In [20]:
# Define reproducible file paths for export
out_dir = os.path.join(et.io.HOME, 'Downloads')
filename = os.path.join(out_dir, 'linearFit.tif')

# Exports two files - one file per band ('filename.bandname.tif')
geemap.ee_export_image(clipped_linearFit, filename=filename, scale=90, region=roi, file_per_band=True)

Generating URL ...
Please wait ...
Data downloaded to /Users/richardudell/Downloads


In [21]:
# Confirm that linearFit.scale.tif appears in your Downloads folder
scale_path = os.path.join(et.io.HOME, 'Downloads', 'linearFit.scale.tif')

# Import scale (slope values for each pixel in the ROI)
with rio.open(scale_path) as src:
    scale_tif = src.read()

In [22]:
# Calculate the total number of pixels in the dataset
total = scale_tif < np.inf

# Interpret the slope values as either trending positive or negative (above
# or below zero)
above = scale_tif < 0
below = scale_tif > 0

# Confirm that there are no values at zero
at = scale_tif == 0

print('Number of pixels above zero: ', above.sum(),
      '\nBelow zero: ', below.sum(), 
      '\nAt zero: ', at.sum())

Number of pixels above zero:  21039 
Below zero:  145896 
At zero:  0


In [23]:
# Calculate the total area of the ROI
area_per_pixel = clipped_linearFit.pixelArea()
total_area_roi = area_per_pixel.reduceRegion(ee.Reducer.sum(), roi, 30)
total_area_roi = total_area_roi.getInfo()['area']

In [24]:
# Calculate the percent of pixels trending towards urban
trending_urban_perc = round(above.sum()/total.sum(), 2)

print(trending_urban_perc, '% of the pixels in the ROI are trending towards urban.')
print('This is the same as: ', int(trending_urban_perc * total_area_roi), ' square meters, or', )
print(round((trending_urban_perc * total_area_roi)/2.59e6, 8), ' square miles.')

0.13 % of the pixels in the ROI are trending towards urban.
This is the same as:  104614003  square meters, or
40.39150716  square miles.


# Implications/Next Steps
This tutorial stands to prove that the Google Earth Engine API can run effectively in Jupyter Notebooks.  Moving forward, I would like to expand the temporal range of the analysis to 2020.  This would increase the relevance of the identification of urban trend.  However, this would require the addition of Landsat 8 imagery to the workflow and likely the calculation of new endmembers for spectral unmixing.  Additionally, I would like to expand the linear analysis to include a linear regression model that could predict future urban development in the land surrounding DIA.

# Acknowledgements
I would like to thank the Earth Lab at the University of Colorado Boulder for my education in Python and open-source spatial-data-analysis over the past year.  Thank Dr. Jenny Palomino for directly advising me on this project.  Thank you to the director of Earth Lab, Dr. Leah Wasser.  I would also like to thank Joe McGlinchy at Earth Lab for help understanding hyperspectral analysis.  Thank you to Kari Klein, Stephane Belmon, and Brian Shucker at Google for early direction with this project.